# Patch pooling (10pts)
Your task is to implement patch pooling. Patch pooling takes an input sequence $(a_0, a_1, \ldots, a_{B-1})$ of $D$-dimensional embeddings and output an output sequence $(b_0, \ldots, b_{k-1})$ of $D$-dimensional embeddings. The length of the output sequence is not longer than the length of the input sequence and is bounded by $P$. Each element of the input sequence is called a token. Each element of the output sequence is called a patch. Consecutive patches are constructed as a mean pooling of consecutive contiguous token spans.

You are given two tensors:
1. `batch` - a $3$-dimensional tensor, which is an input to a standard transformer model with the following dimensions:
* B - batch size
* S - sequence lenght
* D - dimension of embedding of a single token

`batch[x,y,:]` is the embedding of the $y+1$-th token of the $x+1$-th sequence in the `batch`.

2. `patch_lengths` - $2$-dimensional integer-valued tensor with the following dimensions:
* B - batch size
* P - maximal number of patches

`patch_lengths[x,y]` is the number of tokens forming patch number $y+1$ in the $x+1$-th sequence in the `batch`.

The output should be a $3$-dimensional tensor with batch of sequences of patch embeddings.

# Example
The following snippet
```python
batch = torch.tensor([[[ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 2.,  2.,  2.,  2.,  2.],
         [ 3.,  3.,  3.,  3.,  3.],
         [ 3.,  3.,  3.,  3.,  3.]],

        [[ 4.,  4.,  4.,  4.,  4.],
         [ 4.,  4.,  4.,  4.,  4.],
         [ 4.,  4.,  4.,  4.,  4.],
         [ 4.,  4.,  4.,  4.,  4.],
         [ 5.,  5.,  5.,  5.,  5.],
         [-1., -1., -1., -1., -1.]],

        [[ 6.,  6.,  6.,  6.,  6.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.]]])
patch_lengths = torch.tensor([[3, 1, 2],
        [4, 1, 0],
        [1, 0, 0]])
patch_pooling = PatchPooling()
output = patch_pooling(batch, patch_lengths)
output
```

should ouptut

```python
torch.tensor([[[1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2.],
         [3., 3., 3., 3., 3.]],

        [[4., 4., 4., 4., 4.],
         [5., 5., 5., 5., 5.],
         [-1., -1., -1., -1., -1.]],

        [[6., 6., 6., 6., 6.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.]]])
```

Remarks:

1. In this problem you can assume that embeddings of the padding token are vectors with all coordinates equal to $-1$.

2. Solutions will be graded with unit tests. You are given a single test case, which will be a part of evaluation.

3. Solutions not satisfying the below requirements will be graded up to 4pts:
* You are not allowed to call custom python functions
* You are not allowed to use Python loops
* Your are not allowed to use any other imports

In [2]:

import pytest
import torch


class PatchPooling(torch.nn.Module):
    def forward(self, batch: torch.Tensor, patch_lengths: torch.Tensor) -> torch.Tensor:
        B, S, D = batch.shape
        B_1, P = patch_lengths.shape

        assert B == B_1

        ### Your code goes here ###

        patch_ends = torch.cumsum(input = patch_lengths, dim = 1)
        patch_beginings = patch_ends-patch_lengths
        indexes = torch.arange(0, S)

        mask = (indexes.view((1, 1, -1)) < patch_ends.view((B, P, 1))) & \
            (indexes.view((1, 1, -1)) >= patch_beginings.view((B, P, 1)))

        patch_sum = torch.matmul(mask, batch)

        patch_lengths = torch.where(patch_lengths==0, 1, patch_lengths).unsqueeze(-1)
        safe_lengths = patch_lengths.clamp(min=1.0)
        result = patch_sum/safe_lengths
        result = torch.where(patch_lengths > 0, result, -1*torch.ones_like(result))

        return result

        ###########################

        


class TestPatchPooling:
    @pytest.mark.parametrize(
        "batch,patch_lengths,expected_output",
        [
            (
                torch.tensor(
                    [
                        [
                            [1.0, 1.0, 1.0, 1.0, 1.0],
                            [1.0, 1.0, 1.0, 1.0, 1.0],
                            [1.0, 1.0, 1.0, 1.0, 1.0],
                            [2.0, 2.0, 2.0, 2.0, 2.0],
                            [3.0, 3.0, 3.0, 3.0, 3.0],
                            [3.0, 3.0, 3.0, 3.0, 3.0],
                        ],
                        [
                            [4.0, 4.0, 4.0, 4.0, 4.0],
                            [4.0, 4.0, 4.0, 4.0, 4.0],
                            [4.0, 4.0, 4.0, 4.0, 4.0],
                            [4.0, 4.0, 4.0, 4.0, 4.0],
                            [5.0, 5.0, 5.0, 5.0, 5.0],
                            [-1.0, -1.0, -1.0, -1.0, -1.0],
                        ],
                        [
                            [6.0, 6.0, 6.0, 6.0, 6.0],
                            [-1.0, -1.0, -1.0, -1.0, -1.0],
                            [-1.0, -1.0, -1.0, -1.0, -1.0],
                            [-1.0, -1.0, -1.0, -1.0, -1.0],
                            [-1.0, -1.0, -1.0, -1.0, -1.0],
                            [-1.0, -1.0, -1.0, -1.0, -1.0],
                        ],
                    ]
                ),
                torch.tensor([[3, 1, 2], [4, 1, 0], [1, 0, 0]]),
                torch.tensor(
                    [
                        [
                            [1.0, 1.0, 1.0, 1.0, 1.0],
                            [2.0, 2.0, 2.0, 2.0, 2.0],
                            [3.0, 3.0, 3.0, 3.0, 3.0],
                        ],
                        [
                            [4.0, 4.0, 4.0, 4.0, 4.0],
                            [5.0, 5.0, 5.0, 5.0, 5.0],
                            [-1.0, -1.0, -1.0, -1.0, -1.0],
                        ],
                        [
                            [6.0, 6.0, 6.0, 6.0, 6.0],
                            [-1.0, -1.0, -1.0, -1.0, -1.0],
                            [-1.0, -1.0, -1.0, -1.0, -1.0],
                        ],
                    ]
                ),
            )
        ],
    )
    def test_forward(
        self,
        batch: torch.Tensor,
        patch_lengths: torch.Tensor,
        expected_output: torch.Tensor,
    ) -> None:
        # given
        patch_pooling = PatchPooling()

        # when
        output = patch_pooling(batch=batch, patch_lengths=patch_lengths)

        # then
        assert torch.all(torch.isclose(output, expected_output))

In [3]:
batch = torch.tensor([
    [[0.0], [0.0]]  # Sequence 0: Two tokens, both are 0.0
])

# Lengths: Patch 1 has length 2.
patch_lengths = torch.tensor([[2]])

In [4]:
B, S, D = batch.shape
B_1, P = patch_lengths.shape

assert B == B_1

        ### Your code goes here ###

patch_ends = torch.cumsum(input = patch_lengths, dim = 1)
patch_beginings = patch_ends-patch_lengths
indexes = torch.arange(0, S)

mask = (indexes.view((1, 1, -1)) < patch_ends.view((B, P, 1))) & \
    (indexes.view((1, 1, -1)) >= patch_beginings.view((B, P, 1)))

mask = mask.to(batch.dtype)

patch_sum = torch.matmul(mask, batch)

patch_lengths = torch.where(patch_lengths==0, 1, patch_lengths).unsqueeze(-1)
safe_lengths = patch_lengths.clamp(min=1.0)
result = patch_sum/safe_lengths
result = torch.where(patch_lengths > 0, result, -1*torch.ones_like(result))

print(result)


tensor([[[0.]]])


In [5]:
!python -m pytest test_patch_pooling.py

============================= test session starts =============================
platform win32 -- Python 3.10.19, pytest-9.0.2, pluggy-1.5.0
rootdir: c:\Users\Julka\GSN-materials\2024-25
collected 1 item

test_patch_pooling.py F                                                  [100%]

================================== FAILURES ===================================
____ TestPatchPooling.test_forward[batch0-patch_lengths0-expected_output0] ____

self = <test_patch_pooling.TestPatchPooling object at 0x000001DF7CAD4E80>
batch = tensor([[[ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [...1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.]]])
patch_lengths = tensor([[3, 1, 2],
        [4, 1, 0],
        [1, 0, 0]])
expected_output = tensor([[[ 1.,  1.,  1.,  1.,  1.],
         [ 2.,  2.,  2.,  2.,  2.],
         [ 3.,  3.,  3.,  3.,  3.]],

        ..., -1.]],

       